<a href="https://colab.research.google.com/github/NathVM/GA/blob/main/Neo4JGraph.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Measuring performance of Graph Analytics Algorithms using Neo4j graphs

---



Imports:

---



In [1]:
import pandas as pd
from py2neo import Graph, Node, Relationship
from neo4j import GraphDatabase
from google.colab import drive

Setup:

---



In [2]:
drive.mount('/content/drive')

!tar -xf /content/drive/MyDrive/Dataset/share/GA/neo4j.tar  # or --strip-components=1
!mv neo4j-community-3.5.8 nj
# disable password, and start server
!sed -i '/#dbms.security.auth_enabled/s/^#//g' nj/conf/neo4j.conf
!nj/bin/neo4j start

!pip install py2neo

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
mv: cannot move 'neo4j-community-3.5.8' to 'nj/neo4j-community-3.5.8': Directory not empty
WARNING! You are using an unsupported Java runtime. 
* Please use Oracle(R) Java(TM) 8, OpenJDK(TM) or IBM J9 to run Neo4j.
* Please see https://neo4j.com/docs/ for Neo4j installation instructions.
Active database: graph.db
Directories in use:
  home:         /content/nj
  config:       /content/nj/conf
  logs:         /content/nj/logs
  plugins:      /content/nj/plugins
  import:       /content/nj/import
  data:         /content/nj/data
  certificates: /content/nj/certificates
  run:          /content/nj/run
Neo4j is already running (pid 71565).
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Neo4j connection:

---



In [3]:
graph = Graph("bolt://localhost:7687")
driver = GraphDatabase.driver("bolt://localhost:7687")

Dataset: 

https://networkrepository.com/TWITTER-Real-Graph-Partial.php

Shared in the google drive 

In [4]:
# Map the shared folder 
# https://drive.google.com/drive/folders/113gZK1io1MZGogAULYoBdrlEUHyJcxRh?usp=sharing 
# to your google drive and modify the file path accordingly
file = "/content/drive/MyDrive/Dataset/share/GA/TWITTER-Real-Graph-Partial.edges"
df = pd.read_csv(file)
df.rename(columns = {'1':'source', '2':'target'}, inplace = True)
print(df.head(5))
dft = df

   source  target
0       2       1
1       3       4
2       4       3
3       3       2
4       2       3


Create Graph :

In [5]:
#Please comment the below line to execute the cell
%%script echo skipping
query = """
WITH $rows AS rows
UNWIND rows AS row
MERGE (source:Node {id: row.source})
MERGE (target:Node {id: row.target})
MERGE (source)-[:CONNECTS_TO]-(target)
"""

# set batch size and index properties
batch_size = 1000
index_properties = ['id']

# create indexes on node properties
with driver.session() as session:
    for property_name in index_properties:
        session.run(f"CREATE INDEX ON :Node({property_name})")

# execute the query in batch transactions
with driver.session() as session:
    for i in range(0, len(dft), batch_size):
        batch = dft[i:i+batch_size].to_dict('records')
        session.run(query, rows=batch)

Path Analytics: 

---



In [10]:
query = """
MATCH (source:Node {id: 357908})
MATCH (destination:Node)
WHERE source <> destination
MATCH path = allshortestPaths((source)-[:CONNECTS_TO*]-(destination))
WITH source, destination, reduce(distance = 0, r in relationships(path) | distance + 1) AS distance, nodes(path) AS nodes
RETURN source.id, destination.id, distance, nodes, COLLECT( DISTINCT nodes)
"""
result = graph.run(query)

for record in result:
  nodes = record["nodes"]
  print ([node["id"] for node in nodes])

[357908, 357909]
[357908, 357911]
[357908, 357910]


Centrality Analytics :

---



Community Analytics :

---

